In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
# parameters for generation

in_channels = 3
out_channels = 2
kernel_size = 4
stride = 3
padding = 0

batch_size = 1
x_rows = 7
x_cols = 7

_params_ = ['in_channels', 'out_channels', 'kernel_size', 'stride', 'padding', 'batch_size', 'x_rows', 'x_cols']
print("const size_t {};".format(", ".join([f"{k} = {globals()[k]}" for k in _params_])))

const size_t in_channels = 3, out_channels = 2, kernel_size = 4, stride = 3, padding = 0, batch_size = 1, x_rows = 7, x_cols = 7;


In [3]:
k = nn.Conv2d(in_channels, out_channels, kernel_size, stride, bias=False)
x = torch.randn(batch_size, in_channels, x_rows, x_cols)

In [4]:
k.weight.data = torch.round(k.weight * 100)
x = torch.round(x * 10)

In [5]:
def testcase_codegen(mat, name, dtype=None):
    if isinstance(mat, nn.Parameter):
        mat = mat.detach()
    elif isinstance(mat, torch.Tensor):
        pass
    c_prefix = f"static const elem_t {name}"
    c_dim = "".join([f'[{i}]' for i in mat.shape])
    c_data = " = {};".format(str((mat.type(dtype) if dtype else mat).tolist()).replace('[', '{').replace(']', '}'))
    return c_prefix + c_dim + c_data

print("// c-form generated testcase:")
print(testcase_codegen(k.weight, 'convnet_kernel_weight', torch.LongTensor))
print(testcase_codegen(x, 'convnet_x', torch.LongTensor))

// c-form generated testcase:
static const elem_t convnet_kernel_weight[2][3][4][4] = {{{{-3, 10, 6, -10}, {11, 6, 9, 3}, {-12, -11, -1, -11}, {-2, 10, 10, 8}}, {{-12, -3, 0, -2}, {8, -14, 13, 3}, {-10, 3, 10, 0}, {-1, -1, -12, -3}}, {{4, -2, -1, 10}, {10, 14, -6, 0}, {-11, -3, -9, -7}, {-2, -1, -3, -2}}}, {{{0, -10, 0, 0}, {4, 11, -8, 9}, {-1, 9, 5, -11}, {-12, -13, -5, -9}}, {{-11, 4, -8, -8}, {9, -12, -5, 1}, {-8, -2, 11, -11}, {3, -10, -13, 1}}, {{-3, 5, -5, -3}, {3, 14, 6, 8}, {2, 8, 5, -6}, {-2, 3, 3, 0}}}};
static const elem_t convnet_x[1][3][7][7] = {{{{-2, -13, 4, 0, -25, 4, 5}, {18, 7, -23, 7, 13, -6, 1}, {10, -10, -4, 7, -14, 4, 0}, {24, -19, 5, 8, -12, -11, -3}, {10, -5, -12, -6, -14, -13, 2}, {8, -4, 9, -5, -16, 15, -2}, {-10, -5, -13, -7, -4, -15, 17}}, {{-4, -9, -3, -5, -11, -1, 17}, {0, -1, -10, -4, -9, 16, 5}, {16, -3, 1, 12, 1, 3, 1}, {-4, -8, 0, 14, 19, -17, -3}, {-3, -1, -4, 20, -23, -22, -2}, {-12, -3, -23, -1, -4, 13, 6}, {17, 10, -10, -9, -12, 1, 13}}, {{15, 13, 

In [6]:
print("// results of conv2d(kernel=k, input=x):")
print(k(x))

// results of conv2d(kernel=k, input=x):
tensor([[[[-268.,   25.],
          [  29.,  148.]],

         [[ 595.,  501.],
          [ 699., 1313.]]]], grad_fn=<ConvolutionBackward0>)
